## feeding the trained grid with new test data to generate "fake data"

In [1]:
import os
import json
from argparse import ArgumentParser
from re import split
import numpy as np
from tqdm import tqdm
import imageio
from PIL import Image
np.random.seed(0)

import jax
import jax.numpy as jnp
import plenoxel
from jax.lib import xla_bridge
import json
import random
import copy
import numpy as np
import math
import matplotlib.pyplot as plt

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"


ModuleNotFoundError: No module named 'plenoxel'

Pick a random test view then projected on a 2D grid

In [ ]:
# json file to changed coordinate aka the new training coordinate
file = open('/home/vitran/plenoxels/blender_data/lego/transforms_test.json')
data = json.load(file)
data_gen = copy.deepcopy(data)
frames = []

#taking 10th position
view_sample = data['frames'][10]

#making 2d grid
grid2d = np.zeros((10,10))
x_linspace = np.linspace(start = -0.5, stop = 0.5, num = 10)

for i in x_linspace:
    for j in x_linspace:
        sample = copy.deepcopy(view_sample)
        sample['transform_matrix'][0][-1] += i
        sample['transform_matrix'][1][-1] += j
        print(i,j, sample['transform_matrix'][0][-1],sample['transform_matrix'][1][-1] )
        frames.append(sample)

data_gen['frames'] = frames#json.dumps(frames)
        
file.close()

with open('/home/vitran/plenoxels/blender_data/lego/transforms_test_gen3.json','w') as f:
    json.dump(data_gen, f)

### Feed the projected coordinates to the trained model to get the images for each view

Helper functions copy from plenoptimize

In [ ]:
def get_data(root, stage):
    all_c2w = []
    all_gt = []

    data_path = os.path.join(root, stage)
    data_json = os.path.join(root, 'transforms_' + stage + '.json')
    print('LOAD DATA', data_path)
    j = json.load(open(data_json, 'r'))

    for frame in tqdm(j['frames']):
        fpath = os.path.join(data_path, os.path.basename(frame['file_path']) + '.png')
        c2w = frame['transform_matrix']
#         im_gt = imageio.imread(fpath).astype(np.float32) / 255.0
#         im_gt = im_gt[..., :3] * im_gt[..., 3:] + (1.0 - im_gt[..., 3:])
        all_c2w.append(c2w)
#         all_gt.append(im_gt)
    img0 = imageio.imread('/home/vitran/plenoxels/blender_data/lego/test/r_0.png')
    all_gt.append(img0)
    focal = 0.5 * all_gt[0].shape[1] / np.tan(0.5 * j['camera_angle_x'])
#     all_gt = np.asarray(all_gt)
    all_c2w = np.asarray(all_c2w)
    return focal, all_c2w, all_gt



def get_loss(data_dict, c2w, gt, H, W, focal, resolution, radius, harmonic_degree, jitter, uniform, key, sh_dim, occupancy_penalty, interpolation, nv):
    rays = plenoxel.get_rays(H, W, focal, c2w)
    rgb, disp, acc, weights, voxel_ids = plenoxel.render_rays(data_dict, rays, resolution, key, radius, harmonic_degree, jitter, uniform, interpolation, nv)
    mse = jnp.mean((rgb - lowpass(gt, resolution))**2)
    indices, data = data_dict
    loss = mse + occupancy_penalty * jnp.mean(jax.nn.relu(data[-1]))
    return loss


def get_loss_rays(data_dict, rays, gt, resolution, radius, harmonic_degree, jitter, uniform, key, sh_dim, occupancy_penalty, interpolation, nv):
    rgb, disp, acc, weights, voxel_ids = plenoxel.render_rays(data_dict, rays, resolution, key, radius, harmonic_degree, jitter, uniform, interpolation, nv)
    mse = jnp.mean((rgb - gt)**2)
    indices, data = data_dict
    loss = mse + occupancy_penalty * jnp.mean(jax.nn.relu(data[-1]))
    return loss


def get_rays_np(H, W, focal, c2w):
    i, j = np.meshgrid(np.arange(W, dtype=np.float32) + 0.5, np.arange(H, dtype=np.float32) + 0.5, indexing='xy')
    dirs = np.stack([(i-W*.5)/focal, -(j-H*.5)/focal, -np.ones_like(i)], -1)
    # Rotate ray directions from camera frame to the world frame
    rays_d = np.sum(dirs[..., np.newaxis, :] * c2w[:3,:3], -1)  # dot product, equals to: [c2w.dot(dir) for dir in dirs]
    # Translate camera frame's origin to the world frame. It is the origin of all rays.
    rays_o = np.broadcast_to(c2w[:3,-1], np.shape(rays_d))
    return rays_o, rays_d


def render_pose_rays(data_dict, c2w, H, W, focal, resolution, radius, harmonic_degree, jitter, uniform, key, sh_dim, batch_size, interpolation, nv):
    rays_o, rays_d = get_rays_np(H, W, focal, c2w)
    rays_o = np.reshape(rays_o, [-1,3])
    rays_d = np.reshape(rays_d, [-1,3])
    rgbs = []
    disps = []
    for i in range(int(np.ceil(H*W/batch_size))):
        start = i*batch_size
        stop = min(H*W, (i+1)*batch_size)
        if jitter > 0:
            rgbi, dispi, acci, weightsi, voxel_idsi = jax.lax.stop_gradient(plenoxel.render_rays(data_dict, (rays_o[start:stop], rays_d[start:stop]), resolution, key[start:stop], radius, harmonic_degree, jitter, uniform, interpolation, nv))
        else:
            rgbi, dispi, acci, weightsi, voxel_idsi = jax.lax.stop_gradient(plenoxel.render_rays(data_dict, (rays_o[start:stop], rays_d[start:stop]), resolution, None, radius, harmonic_degree, jitter, uniform, interpolation, nv))
        rgbs.append(rgbi)
        disps.append(dispi)
    rgb = jnp.reshape(jnp.concatenate(rgbs, axis=0), (H, W, 3))
    disp = jnp.reshape(jnp.concatenate(disps, axis=0), (H, W))
    return rgb, disp, None, None


Set parameters and load in the trained plenoxel model

In [ ]:
# PARAMETERS
resolution = 256
lr_rgb = 150 * (resolution ** 1.75)
lr_sigma = 51.5 * (resolution ** 2.37)
harmonic_degree = 2
jiter = 0.0
automatic_lr = True
uniform = 0.5
physical_batch_size = 1000
interpolation = 'trilinear'
nv = True
radius = 1.3
render_interval = 40
jitter = 0.0

# Load the trained model
focal, train_c2w, train_gt = get_data('/home/vitran/plenoxels/blender_data/lego/', "train")
test_focal, test_c2w, _ = get_data('/home/vitran/plenoxels/blender_data/lego/', "test_gen3")
assert focal == test_focal
H, W = train_gt[0].shape[:2]
#     n_train_imgs = len(train_c2w)
n_test_imgs = len(test_c2w)

reload_dir = '/home/vitran/plenoxels/jax_logs/experiment/epoch_0/' #os.path.join(log_dir, f'epoch_{FLAGS.reload_epoch}')
print(f'Reloading the grid from {reload_dir}')
data_dict = plenoxel.load_grid(dirname=reload_dir, sh_dim = (harmonic_degree + 1)**2)

Feed coordinates to the model to output image of the view

In [ ]:
def run_test_step(i, data_dict, test_c2w, test_gt, H, W, focal, FLAGS, key, name_appendage=''):
    print('Evaluating')
    sh_dim = (harmonic_degree + 1)**2
    tpsnr = 0.0
    for j, (c2w, _) in tqdm(enumerate(zip(test_c2w, test_gt))):
        print(c2w[1].shape)
        print(c2w)
        rgb, disp, _, _ = render_pose_rays(data_dict, c2w, H, W, focal, resolution, radius, harmonic_degree, jitter, uniform, key, sh_dim, physical_batch_size, interpolation, nv)
#         mse = jnp.mean((rgb - gt)**2)
#         psnr = -10.0 * np.log(mse) / np.log(10.0)
#         tpsnr += psnr

#         if render_interval > 0 and j % render_interval == 0:
#             disp3 = jnp.concatenate((disp[...,jnp.newaxis], disp[...,jnp.newaxis], disp[...,jnp.newaxis]), axis=2)
#             vis = jnp.concatenate((gt, rgb, disp3), axis=1)
        print(j)
        vis = np.asarray((rgb * 255)).astype(np.uint8)
        imageio.imwrite(f"/home/vitran/plenoxels/jax_logs/lego_generate3/{j:04}_{i:04}{name_appendage}.png", vis)
#         del rgb, disp
#     tpsnr /= n_test_imgs
#     return tpsnr
keys = jax.vmap(jax.random.PRNGKey, in_axes=0, out_axes=0)(jnp.arange(physical_batch_size))
test_gt = np.ones_like(test_c2w)
run_test_step(0, data_dict, test_c2w, test_gt, H, W, focal, None, None)


Model output

In [ ]:
key = jax.vmap(jax.random.PRNGKey, in_axes=0, out_axes=0)(jnp.arange(physical_batch_size))
sh_dim = (harmonic_degree + 1)**2
c2w = test_c2w[0]
rgb, disp, _, _ = render_pose_rays(data_dict, c2w, H, W, focal, resolution, radius, harmonic_degree, jitter, uniform, key, sh_dim, physical_batch_size, interpolation, nv)
plt.imshow(rgb)

## Reformating the fake training json file

In [ ]:
# change images name
import os
basedir = "/home/vitran/plenoxels/blender_data/lego_gen3/train/"
for file in os.listdir(basedir):
    if file[-3:]=='png':
        cnt = int(file[2:4])
        newname = "r_"+str(cnt)
        os.rename(basedir+file, basedir+f"r_{str(cnt)}.png")

Assigning image view with the corresponding coordinate

In [ ]:
file = open('/home/vitran/plenoxels/blender_data/lego_gen3/transforms_test.json')
data = json.load(file)
cnt = 0
for img in data['frames']:
    img['file_path'] = './train/r_'+str(cnt)
    cnt+=1
    a = img['file_path']
    print(a)

# data_gen['frames'] = frames#json.dumps(frames)
        
file.close()

# with open('/home/vitran/plenoxels/blender_data/lego_gen3/transforms_train.json','w') as f:
#     json.dump(data, f)

Debugging missing alpha channel from the input data

In [ ]:
data_path = '/home/vitran/plenoxels/blender_data/lego_gen3/'
data_json = '/home/vitran/plenoxels/blender_data/lego_gen3/transforms_train.json'

# data_path = '/home/vitran/plenoxels/blender_data/lego/'
# data_json = '/home/vitran/plenoxels/blender_data/lego/transforms_train.json'

j = json.load(open(data_json, 'r'))
for frame in j['frames']:
    fpath = os.path.join(data_path+'train/', os.path.basename(frame['file_path']) + '.png')
    im_gt = imageio.imread(fpath).astype(np.float32) / 255.0
    plt.imshow(im_gt[:,:,0])
    plt.show()
#     print('shape here', im_gt.shape)